In [1]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns 
%matplotlib inline

import warnings
warnings.filterwarnings("ignore")

In [2]:
item = pd.read_csv('../data/Antai_AE/Antai_AE_round2_item_attr_20190813.csv')
train = pd.read_csv('../data/Antai_AE/Antai_AE_round2_train_20190813.csv')
test = pd.read_csv('../data/Antai_AE/Antai_AE_round2_test_20190813.csv')

In [4]:
item.isnull().any()

item_id       False
cate_id       False
store_id      False
item_price    False
dtype: bool

In [5]:
train.head()

,country_id,buyer_admin_id,item_id,log_time,irank,buy_flag
0,xx,1,7554,2018-04-19 10:59:56,79,0
1,xx,1,7937,2018-04-19 11:17:45,62,0
2,xx,1,7544,2018-04-18 22:19:30,81,0
3,xx,1,7559,2018-04-18 22:15:27,82,0
4,xx,1,7554,2018-04-19 11:28:34,46,0


In [6]:
test.head()

,country_id,buyer_admin_id,item_id,log_time,irank,buy_flag
0,zz,186,5759164,2018-04-16 05:11:47,37,0
1,zz,186,2321601,2018-04-16 04:57:35,48,0
2,zz,186,5244747,2018-04-17 12:24:43,2,0
3,zz,186,2136020,2018-04-17 11:53:41,8,0
4,zz,186,2137602,2018-04-16 04:57:36,47,0


In [8]:
train["log_time"].min(), train["log_time"].max()

('2018-04-16 00:00:00', '2018-04-30 23:59:00')

In [9]:
test["log_time"].min(), test["log_time"].max()

('2018-04-16 00:00:05', '2018-04-30 23:52:06')

In [10]:
train["country_id"].unique(), test["country_id"].unique()

(array(['xx', 'zz', 'yy'], dtype=object), array(['zz', 'yy'], dtype=object))

In [11]:
train["irank"].min(), test["irank"].min()

(1, 2)

In [12]:
train["buyer_admin_id"].nunique(), test["buyer_admin_id"].nunique()

(614960, 9844)

In [13]:
train["irank"].max(), test["irank"].max()

(264473, 39848)